In [1]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import warnings
import pprint
import time
warnings.simplefilter('ignore')

#ファイルのインポート
import evaluation as ev
import control_structure as cs
import race
import table as tb
import init_table as init
import creating_test as test
import os_operation as _os_

RESULTS_FILE_NAME='sample' #.csvつけない

# インポートする dir
path='data/'

# エクスポートする dir
results_path='results_buffer2/'


#files=_os_.get_files(path)
#send_files=files['send_files']
#recv_files=files['recv_files']


heading_results=('Task','Buffer','TestCase','SVAR','Def-Use','Plnv')
heading=('Thread','Port','Event','Index')
pre_results=pd.DataFrame({},columns=heading_results)
new_results=pd.DataFrame({},columns=heading_results)

RECEIVE_SHEET_NAME=path+'recv-task-2-buffer-2.csv'
SEND_SHEET_NAME=path+'send-task-2-buffer-2.csv'

RECEIVE_SHEET_NAME=path+'res.csv'
SEND_SHEET_NAME=path+'sen.csv'


Qs=[pd.DataFrame({})]
Qr=[pd.DataFrame({})]
Qr[0]=pd.read_csv(RECEIVE_SHEET_NAME,names=heading)
Qs[0]=pd.read_csv(SEND_SHEET_NAME,names=heading)
init_tables=init.init_main(Qr,Qs)
Qr=init_tables['Qr']
Qs=init_tables['Qs']
Qr_unique=init_tables['Qr_unique']
Qs_unique=init_tables['Qs_unique']

for way in range(2):
    if way==0:
        t_way=2
        combination=0 #0の場合は既存研究の方法、1の場合は組み合わせを考慮した手法
        ev_table=pre_results
    elif way==1:
        t_way=2
        combination=1
        ev_table=new_results
        Qr=Qr[:1]
        Qs=Qs[:1]
    else:
        print('プログラムを終了します') # shoudn't reach here
        break


    number=0 # 実験の回数
    check_digit=0

    race_set={}
    

    start = time.time()
    print("Creating race set",end='')
    race_set=race.creating_race_set(Qr,Qs,Qs_unique,race_set,combination)
    elapsed_time = time.time() - start
    print ("\nCreating race set took:{:.4g}".format(elapsed_time) + "[sec]")
    #pprint.pprint((race_set))


    start = time.time()
    print("Creating race table",end='')
    table=tb.construct_race_table(Qs,Qr,Qs_unique,Qr_unique,race_set,t_way)
    if t_way>1:
        table=tb.expand_table(Qr,Qs,race_set,table,t_way)
    elapsed_time = time.time() - start
    print ("\nCreating race table took:{:.4g}".format(elapsed_time) + "[sec]")
    table=table.astype('int64')

    check_digit=1

    start = time.time()
    results=test.create_new_testcase(number,check_digit,Qr,Qs,table,race_set,Qr_unique,Qs_unique)
    Qr=results['recv']
    Qs=results['send']
    elapsed_time = time.time() - start
    print ("Creating test case took:{:.4g}".format(elapsed_time) + "[sec]")
    print('The number of Test Case is {}.'.format(len(Qr)))


    evaluation=ev.evaluate(Qr,Qs)
    Q=pd.concat([Qs[0],Qr[0]]).sort_index()
    display(table)
    if way==0:
        pre_results=pre_results.append(evaluation,ignore_index=True)
    elif way==1:
        new_results=new_results.append(evaluation,ignore_index=True)
    else:
        print('error')
        
_os_.save_file(RESULTS_FILE_NAME,results_path,pre_results,new_results)


finish initialize
Creating race set
Creating race set took:0.01702[sec]
Creating race table....Expanding table
..
Creating race table took:0.06901[sec]
...Creating test case took:0.07699[sec]
The number of Test Case is 4.


,r1,r3
0,0,1
1,1,0
2,1,1


Creating race set
Creating race set took:0.03197[sec]
Creating race table....Expanding table
..
Creating race table took:0.08699[sec]
...Creating test case took:0.07904[sec]
The number of Test Case is 4.


,r1,r3
0,0,1
1,1,0
2,1,1


In [2]:
Qs_unique

,ID,Thread,Port,Event,Index,cstruct
0,s1,T1,p2,send,1,
1,s2,T4,p2,send,1,
2,s3,T2,p3,send,3,"[r1, s1, r2, s2]"
3,s4,T4,p3,send,2,[s2]


In [8]:
display(Qr_unique)
display(Qs_unique)

,ID,Thread,Port,Event,Index,cstruct
0,r1,T2,p2,res,1,
1,r2,T2,p2,res,2,"[, s1, r1]"
2,r3,T3,p3,res,1,
3,r4,T3,p3,res,2,"[, r3, s3]"


,ID,Thread,Port,Event,Index,cstruct
0,s1,T1,p2,send,1,
1,s2,T4,p2,send,1,"[, s1, r1]"
2,s3,T2,p3,send,3,
3,s4,T4,p3,send,2,"[, r3, s3]"


In [5]:
display(Q)

,ID,Thread,Port,Event,Index,KEY
0,s1,T4,6470180,send,1,T4-1
0,r1,T2,6470180,recv,1,T2-1
1,s2,T3,6470180,send,1,T3-1
1,r2,T1,6470180,recv,1,T1-1
2,s3,T4,6472276,send,2,T4-2
2,r3,T2,6472276,recv,2,T2-2
3,r4,T1,6472276,recv,2,T1-2
3,s4,T3,6472276,send,2,T3-2
4,r5,T2,6470180,recv,3,T2-3
4,s5,T4,6470180,send,3,T4-3


In [29]:
print(combination_table,port,ID,cand_send_event.ID,pair_event.ID,cand_recv_event.ID)

         PORT     SEND     RECV
0    14924324  [T1,T2]  [T4,T3]
1    14924324  [T1,T2]  [T4,T3]
2    14924324  [T2,T1]  [T3,T4]
3    14924324  [T2,T1]  [T3,T4]
4    14924324  [T1,T2]  [T4,T3]
..        ...      ...      ...
113  14924324  [T1,T2]  [T4,T3]
114  14924324  [T1,T2]  [T4,T3]
115  14924324  [T1,T2]  [T4,T3]
116  14924324  [T1,T2]  [T4,T3]
117  14924324  [T1,T2]  [T4,T3]

[118 rows x 3 columns] 14924324 s20 s19 r20 r19


In [24]:
#send1 in str_to_list(check.SEND)
#'T1' in str_to_list(combination_table.iloc[0].SEND)[0]
combination_table.iloc[0].PORT==14924324

False

In [ ]:
{'r1': ['s20', 's12', 's16', 's2', 's8', 's14', 's4', 's6', 's18', 's10', 's3'], 
 'r2': ['s11', 's17', 's9', 's7', 's4', 's13', 's15', 's3', 's19', 's5'],
 'r3': ['s20', 's12', 's16', 's8', 's14', 's4', 's6', 's18', 's10', 's5'], 
 'r4': ['s11', 's17', 's9', 's7', 's13', 's6', 's15', 's19', 's5'], 
 'r5': ['s20', 's12', 's16', 's8', 's7', 's14', 's6', 's18', 's10'], 
 'r6': ['s11', 's17', 's9', 's8', 's13', 's7', 's15', 's19'], 
 'r7': ['s20', 's12', 's16', 's9', 's8', 's14', 's18', 's10'], 
 'r8': ['s11', 's17', 's9', 's13', 's15', 's10', 's19'], 
 'r9': ['s11', 's20', 's12', 's16', 's14', 's18', 's10'], 
 'r10': ['s11', 's17', 's12', 's13', 's15', 's19'], 
 'r11': ['s20', 's12', 's16', 's14', 's13', 's18'], 
 'r12': ['s17', 's14', 's13', 's15', 's19'], 
 'r13': ['s20', 's16', 's14', 's15', 's18'], 
 'r14': ['s16', 's15', 's19', 's17'], 
 'r15': ['s16', 's20', 's18', 's17'], 
 'r16': ['s19', 's18', 's17'], 
 'r17': ['s20', 's19', 's18'], 
 'r18': ['s19', 's20'], 
 'r19': ['s20'], 
 'r20': []}

{'r1': ['s16', 's14', 's4', 's20', 's2', 's10', 's12', 's18', 's6', 's8', 's3'],
 'r2': ['s5', 's19', 's11', 's15', 's4', 's17', 's7', 's9', 's13', 's3'],
 'r3': ['s5', 's16', 's14', 's4', 's20', 's10', 's12', 's18', 's6', 's8'],
 'r4': ['s5', 's19', 's11', 's15', 's17', 's7', 's9', 's6', 's13'],
 'r5': ['s16', 's14', 's7', 's20', 's10', 's12', 's18', 's6', 's8'],
 'r6': ['s19', 's11', 's15', 's17', 's7', 's9', 's8', 's13'],
 'r7': ['s16', 's14', 's20', 's12', 's10', 's18', 's9', 's8'],
 'r8': ['s19', 's11', 's15', 's17', 's10', 's9', 's13'],
 'r9': ['s16', 's14', 's11', 's20', 's12', 's10', 's18']
  'r10': ['s19', 's11', 's15', 's17', 's12', 's13'],
 'r11': ['s16', 's14', 's20', 's12', 's18', 's13'],
 'r12': ['s19', 's14', 's15', 's17', 's13'],
 'r13': ['s16', 's14', 's15', 's20', 's18'],
 'r14': ['s17', 's16', 's19', 's15'],
 'r15': ['s17', 's16', 's20', 's18'],
 'r16': ['s17', 's19', 's18'],
 'r17': ['s20', 's19', 's18'],
 'r18': ['s20', 's19'],
 'r19': ['s20'],
 'r20': [],